## Fraction-free algorithm for [n-1, n] scalar Padé Approximation

A discret one-channel signal $s_0, s_1, \ldots,s_k,\ldots$ has assocatied a signal generating series:
$$ S(z) = s_0 + s_1 z + \cdots + s_k z^k + \cdots$$

The poles, residues and zeros of the Padé rational approximant of $S(z)$ are used to characterize the signal. 

There are several algorithms available for calculating the Padé approximation. Many algorithms are unstable numerically because of loss of precision, specially for large orders. The following procedure is efficient and preserves the accuracy by using exact integer algebra by avoiding fractions. The cost of this benefit is that the numbers involved increase by few digits at each iteration, exceeding the available range of standard 4-byte integers available in computer hardware. However, integers in Python are not limited in size, and integer operations for `BigInt` are done quickly in hardware when possible, and much slower (by a factor of hundreds) in software when necessary. 

Define a set of polynomials by recursion;

$$q_n(z) = e_{n-1} q_{n-1}(z) - z e_n  q_{n-2}(z)$$

where $e_n$ is the coefficient of $z^n$ in the product $q_{n-1}(z) S(z)$

$$e_n = \text{Coeff}(q_{n-1} S(z), z^n)$$

The recursion is started by $q_{-2}=0$, $q_{-1}(z) = 1$, $e_{-1} = 1$.

We use `sympy` to explore this set of polynomials using symbolic algebra, instead of numbers. For small orders the code below calculates polynomials $q_k$ and coefficients $e_k$:

In [6]:
import sympy as sp

In [7]:
s = sp.symbols('s0,s1,s2,s3,s4,s5,s6,s7')
z = sp.symbols('z')
P, S = 3, 0
for k in range(2*P+1):
    S += s[k]*z**k
q,e = {},{}
q[-2], q[-1], e[-1] = 0, 1, 1
for k in range(2*P+1):
    e[k] = ((S*q[k-1]).expand()).coeff(z,k)
    q[k] = (e[k-1]*q[k-1] - z*e[k]*q[k-2]).expand()

**Point #1:** The degrees of of polynomials increase by one every other step

In [8]:
[sp.degree(q[k], gen=z) for k in range(2*P+1)]

[0, 1, 1, 2, 2, 3, 3]

This means that the degree of $q_k$ is half of order of $q_k$ plus one: $\text{deg}(q_k(z)) = \text{Floor}((k+1)/2)$

**Point #2:** Each polynomial $q_k$ is exactly divisible by $e_{k-2}$

For example $q_3$ is exactly divisible by $e_1$

In [9]:
(q[3]/e[1]).expand()

s0**2*s2 - s0**2*s3*z - s0*s1**2 + s0*s1*s2*z + s0*s1*s3*z**2 - s0*s2**2*z**2

and  $q_4$ is exactly divisible by $e_2$, and  by $e_1$

In [10]:
(q[4]/e[2]/e[1]).simplify()

s0**2*(s1*s3 - s1*s4*z - s2**2 + s2*s3*z + s2*s4*z**2 - s3**2*z**2)

This means that a more efficient recursion, which keeps the growth of the coefficients of the polynomials under control, can be defined as

$$q_n(z) = \frac{e_{n-1}}{e_{n-2}} q_{n-1}(z) - z \frac{e_n}{e_{n-2}}  q_{n-2}(z)$$

The case when $e_{n-2}$ is accidentally zero will need to be dealt separately.


We check if this true directly, and adding $e_{-1} = 1$

In [11]:
s = sp.symbols('s0,s1,s2,s3,s4,s5,s6,s7')
z = sp.symbols('z')
P, S = 3, 0
for k in range(2*P+1):
    S += s[k]*z**k
q,e = {},{}
q[-2], q[-1], e[-2], e[-1] = 0,1,1,1
for k in range(2*P+1):
    e[k] = ((S*q[k-1]).expand()).coeff(z,k)
    q[k] = (((e[k-1]*q[k-1]- z*e[k]*q[k-2])/e[k-2]).expand()).simplify()

Indeed, all divisions are exact and the resulting polynomials are fraction free. All polynomials have integer coefficients if the signal have integers, such that there are no error associated with loss of precision due to finite representation of floating point numbers in computers. We can list few polynomials:

In [12]:
q

{-2: 0,
 -1: 1,
 0: 1,
 1: s0 - s1*z,
 2: s1 - s2*z,
 3: s0*s2 - s0*s3*z - s1**2 + s1*s2*z + s1*s3*z**2 - s2**2*z**2,
 4: s1*s3 - s1*s4*z - s2**2 + s2*s3*z + s2*s4*z**2 - s3**2*z**2,
 5: s0*s2*s4 - s0*s2*s5*z - s0*s3**2 + s0*s3*s4*z + s0*s3*s5*z**2 - s0*s4**2*z**2 - s1**2*s4 + s1**2*s5*z + 2*s1*s2*s3 - s1*s2*s4*z - s1*s2*s5*z**2 - s1*s3**2*z + s1*s3*s4*z**2 - s1*s3*s5*z**3 + s1*s4**2*z**3 - s2**3 + s2**2*s3*z + s2**2*s4*z**2 + s2**2*s5*z**3 - s2*s3**2*z**2 - 2*s2*s3*s4*z**3 + s3**3*z**3,
 6: s1*s3*s5 - s1*s3*s6*z - s1*s4**2 + s1*s4*s5*z + s1*s4*s6*z**2 - s1*s5**2*z**2 - s2**2*s5 + s2**2*s6*z + 2*s2*s3*s4 - s2*s3*s5*z - s2*s3*s6*z**2 - s2*s4**2*z + s2*s4*s5*z**2 - s2*s4*s6*z**3 + s2*s5**2*z**3 - s3**3 + s3**2*s4*z + s3**2*s5*z**2 + s3**2*s6*z**3 - s3*s4**2*z**2 - 2*s3*s4*s5*z**3 + s4**3*z**3}

**Point #3:** The coefficient of $z^n$ of $q_{n}(z) S(z)$ is zero. 

Indeed, from recursion we get:

$$\text{Coeff}(q_{n}(z) S(z), z^n) = e_{n-1} \text{Coeff}(q_{n-1} S(z), z^n) - e_n \text{Coeff}(q_{n-2}(z) S(z), z^{n-1}) = e_{n-1} e_n - e_n e_{n-1} = 0$$

This means that the product $q_{n}(z) S(z)$ has the following structure
$$q_n(z)S(z) = p_n(z) + 0 \times z^n + e_{n+1} z^{n+1} + {\cal O}(z^{n+2})$$

with a polynomial $p_n(z)$ defined as the truncation of $q_n(z)$ up to power $z^{n-1}$: $p_n(z) = \text{Trunc}(q_n(z) S(z), z^{n-1})$

**Point #4:** The associated polynomials $p_n(z)$ have the same recursion relation as $q_n(z)$. This can be seen from
$$ 0 = q_n(z) - e_{n-1} q_{n-1}(z) + z e_n  q_{n-2}(z) = p_n(z) - e_{n-1} p_{n-1}(z) + z e_n  p_{n-2}(z) + {\cal O}(z^n)$$

We obtain polynomials $p_k$ by truncating $q_k(z) S(z)$. Since they have exactly the same recursion, the degree of each polynomial $p_k$ also increases by one every second step, but
$\text{Deg}(p_k(z)) = \text{Floor}(k/2)$.


In [13]:
exp = sp.Wild('exp')
p = [(q[k]*S).expand().replace(pow(z,exp), lambda exp: 0 if exp > k else pow(z,exp)) for k in range(2*P+1)]
[sp.degree(p[k], gen=z) for k in range(2*P+1)]

[0, 0, 1, 1, 2, 2, 3]

We list few $p$ polynomials below:

In [14]:
p

[s0,
 s0**2,
 s0*s1 - s0*s2*z + s1**2*z,
 s0**2*s2 - s0**2*s3*z - s0*s1**2 + 2*s0*s1*s2*z - s1**3*z,
 s0*s1*s3 - s0*s1*s4*z - s0*s2**2 + s0*s2*s3*z + s0*s2*s4*z**2 - s0*s3**2*z**2 + s1**2*s3*z - s1**2*s4*z**2 - s1*s2**2*z + 2*s1*s2*s3*z**2 - s2**3*z**2,
 s0**2*s2*s4 - s0**2*s2*s5*z - s0**2*s3**2 + s0**2*s3*s4*z + s0**2*s3*s5*z**2 - s0**2*s4**2*z**2 - s0*s1**2*s4 + s0*s1**2*s5*z + 2*s0*s1*s2*s3 - 2*s0*s1*s2*s5*z**2 - 2*s0*s1*s3**2*z + 2*s0*s1*s3*s4*z**2 - s0*s2**3 + s0*s2**2*s3*z + 2*s0*s2**2*s4*z**2 - 2*s0*s2*s3**2*z**2 - s1**3*s4*z + s1**3*s5*z**2 + 2*s1**2*s2*s3*z - 2*s1**2*s2*s4*z**2 - s1**2*s3**2*z**2 - s1*s2**3*z + 3*s1*s2**2*s3*z**2 - s2**4*z**2,
 s0*s1*s3*s5 - s0*s1*s3*s6*z - s0*s1*s4**2 + s0*s1*s4*s5*z + s0*s1*s4*s6*z**2 - s0*s1*s5**2*z**2 - s0*s2**2*s5 + s0*s2**2*s6*z + 2*s0*s2*s3*s4 - s0*s2*s3*s5*z - s0*s2*s3*s6*z**2 - s0*s2*s4**2*z + s0*s2*s4*s5*z**2 - s0*s2*s4*s6*z**3 + s0*s2*s5**2*z**3 - s0*s3**3 + s0*s3**2*s4*z + s0*s3**2*s5*z**2 + s0*s3**2*s6*z**3 - s0*s3*s4**2*z**2 - 2*

**Point #5:** Probably the most important point of the discussion is that the rational function $R_n(z) = p_n(z)/q_n(z)$ is a Padé  approximation for series $S(z)$.

The order of the approximant is $[\text{Floor}(n/2), \text{Floor}((n+1/2))]$. This is because
$$q_n(z) S(z) - p_n(z) = {\cal O}(z^{n+1})$$

-------------------

We can test this idea for the series resulting from rational function with 2 poles. Let start with
$$ S(z) = \frac{r_1}{p_1 z - 1} + \frac{r_2}{p_2 z - 1}$$
The Taylor expansion is $$S(z) = \sum_{k=0} s_k z^k = \sum_{k=0} (r_1 p_1^k + r_2 p_2^k) z^k$$

The first 4 terms $s_0$, $s_1$, $s_2$, and $s_3$ should be enough to reconstruct the original rational function. In general, for a rational function with $P$ poles, like in the example above with $P=2$, we need the first $2P$ terms $s_0, \ldots, s_{2P-1}$ to generate $q$ polinomials up to $q_{2P-1}$ which has degree $P$, and correspondingly $p$ polynomials up to $p_{2P-1}$ with degree $P-1$. Note that these polynomials have a total of $P+1+P = 2P+1$ unknown coefficients, but the rational approximant $R_{2P-1}(z)$ has exactly $2P$ unknown coefficients, corresponding to the size of the signal.

In [15]:
import sympy as sp
z,r1,p1,r2,p2= sp.symbols('z,r1,p1,r2,p2')
s = {}
P = 2
G = r1/(z*p1-1) + r2/(z*p2-1)
term = sp.series(G,z,0,n=None)
for k in range(2*P):
    s[k]=next(term).coeff(z,k)

In [16]:
S,P = 0,2
q,e = {},{}
for k in range(2*P):
    S += s[k]*z**k
q[-2], q[-1], e[-2], e[-1] = 0,1,1,1
for k in range(2*P):
    e[k] = ((S*q[k-1]).expand()).coeff(z,k)
    q[k] = (((e[k-1]*q[k-1]- z*e[k]*q[k-2])/e[k-2]).expand()).simplify()

Polynomial $q_3$ has degree 2 and its factorization is what we expect.

In [17]:
sp.factor(q[3],gens=z)

r1*r2*(p1*z - 1)*(p2*z - 1)*(p1**2 - 2*p1*p2 + p2**2)

The numerator $p_3$ has degree 1. The rational function $p_3(z)/q_3(z)$ is identical to the original, showing that the procedure works as expected.

In [18]:
exp = sp.Wild('exp')
p = [(q[k]*S).expand().replace(pow(z,exp), lambda exp: 0 if exp > k else pow(z,exp)) for k in range(2*P)]
sp.apart((p[3]/q[3]).simplify(),z)

r1/(p1*z - 1) + r2/(p2*z - 1)

At this point we can obtain the poles and the residues of the rational approximant $R(z)$ by finding the roots of the last $q(z)$ polynomial. The best polynomial root finding algorithms rely on solving an eigenvalue/eigenvector problem for a matrix associtated with the polynomial. Instead, we can go one step further and expose the matrix that is associated with both $p$ and $q$ polynomials through the recurrence equations that will deliver a tridiagonal matrix, making the eigensoler's job even easier.

The degree of odd order polyomials: $q_{-1}$, $q_1$, $q_3$,$\ldots$ increases by one.
It is useful to work only with odd-order polynomials. For that, we neeed to find a double step recursion relation.

In [19]:
import sympy as sp
e = sp.Function('e')
O = sp.Function('O')
k,n,z = sp.symbols('k,n,z')
u = sp.Function('u')

We generate generic polynomial recursion, here $O_n$ can be either $q_n$ or $p_n$, for order $n$, $n+1$, and $n+2$, and use the three equations to remove orders $n-1$ and $n+1$, such that, at the end, we have one recursion that relates orders $n-2$, $n$ and $n+2$, in both cases.

In [20]:
Gn = O(n)-e(n-1)/e(n-2)*O(n-1)+z*e(n)/e(n-2)*O(n-2)
Gn1 = (Gn.replace(n,n+1))
Gn2 = (Gn.replace(n,n+2))
H = (z*Gn*e(n-2)/e(n-1)**2*e(n+1)/e(n) + Gn2/e(n+1) + Gn1/e(n)).expand()
H

z**2*O(n - 2)*e(n + 1)/e(n - 1)**2 + z*O(n)*e(n - 2)*e(n + 1)/(e(n)*e(n - 1)**2) + z*O(n)*e(n + 2)/(e(n)*e(n + 1)) - O(n)/e(n - 1) + O(n + 2)/e(n + 1)

A further simplification is obtained if we define for odd orders $n=2k-1$, $q_n = e_{n-1} u_k z^k$ and $p_n = e_{n-1} v_k z^k$, with $k = 0, 1, 2\ldots, P$ 

In [21]:
H1 =  - H.replace(n, 2*k-1)
H2 = H1.subs(O(2*k-3),u(k-1)*z**(-1)*e(2*k-4)).replace(O(2*k-1),u(k)*e(2*k-2)).replace(O(2*k+1),u(k+1)*z*e(2*k))
H2 = H2.simplify()
H2

-z*e(2*k)*e(2*k - 4)*u(k - 1)/e(2*k - 2)**2 - z*e(2*k)*e(2*k - 3)*u(k)/(e(2*k - 2)*e(2*k - 1)) - z*u(k + 1) - z*e(2*k - 2)*e(2*k + 1)*u(k)/(e(2*k)*e(2*k - 1)) + u(k)

The recursion equation have the following structure: $u_k - z (b_k u_{k-1} + a_k u_k + u_{k+1}) = 0$, where coefficients $a_k$ and $b_k$ are defined as

In [22]:
a = -((H2.coeff(u(k)) - 1)/z).expand()
b = -(H2.coeff(u(k-1))/z).expand()
a

e(2*k)*e(2*k - 3)/(e(2*k - 2)*e(2*k - 1)) + e(2*k - 2)*e(2*k + 1)/(e(2*k)*e(2*k - 1))

In [23]:
b

e(2*k)*e(2*k - 4)/e(2*k - 2)**2

We can verify this, and see it is identical with the recursion equation for $u_k$.

In [24]:
(u(k) - z*(b*u(k-1) + a*u(k) + u(k+1))).expand()

-z*e(2*k)*e(2*k - 4)*u(k - 1)/e(2*k - 2)**2 - z*e(2*k)*e(2*k - 3)*u(k)/(e(2*k - 2)*e(2*k - 1)) - z*u(k + 1) - z*e(2*k - 2)*e(2*k + 1)*u(k)/(e(2*k)*e(2*k - 1)) + u(k)

The first equation for $k=0$ is treated separately. For $q$ polynomials we have 
$$q_0(z) = \frac{e_{-1}}{e_{-2}} q_{-1}(z) \quad and\quad  q_1(z) = \frac{e_0}{e_{-1}} q_0(z) - z \frac{e_1}{e_{-1}} q_{-1}(z)$$
and combining the two initial equations:
$$q_1(z) = \frac{e_{0}}{e_{-2}} q_{-1}(z) - z \frac{e_{1}}{e_{-1}} q_{-1}(z)$$
or
$$ \frac 1{e_{-2}} q_{-1}(z) - z\frac{e_1}{e_0 e_{-1}} q_{-1}(z) - \frac 1{e_{0}} q_{1}(z) = 0$$
but $q_{-1} = u_0 e_{-2}$ and $q_1 = u_1 e_{0} z$, so in terms of the new functions $u_n$ the first equation for $q$ is
$$u_0(z) - z[\frac{e_1 e_{-2}}{e_0 e_{-1}} u_0(z) + u_1(z)] = 0$$
This is consistent with the general recursion equation if $b_0 = 0$ and $a_0 = \frac{e_1 e_{-2}}{e_0 e_{-1}}$

In [25]:
a.replace(k,0)

e(-3)*e(0)/(e(-2)*e(-1)) + e(-2)*e(1)/(e(-1)*e(0))

The first equation $k=0$ is a little bit different for $p_n$ polynomials and $v_k$ functions:

$$p_{1}(z) = \frac{e_0}{e_{-1}} p_0(z) - z \frac{e_1}{e_{-1}} p_{-1}(z)$$ 

or in terms of $v_0$ and $v_1$

$$z v_1(z) = s_0 - z \frac{e_1 e_{-2}}{e_0 e_{-1}} v_0$$

because $p_0 = s_0$ and $e_{-1} = 1$. Since $P_{-1}$ and $v_0$ have to be both zero since $p_1(z) = s_0^2$, we can formally write the $k=0$ equation as
$v_0 - z (a_0 v_0 + v_1) = -s_0$

The whole set of equations, combining $q$ and $p$ equations, is

$$u_0 - z (a_0 u_0 + u_1) = 0$$
$$v_0 - z (a_0 v_0 + v_1) = -s_0$$
$$ \vdots \quad \quad \quad \quad  \vdots \quad \quad \quad \quad \vdots$$
$$u_k - z (b_k u_{k-1} + a_k u_k + u_{k+1}) = 0$$
$$v_k - z (b_k v_{k-1} + a_k v_k + v_{k+1}) = 0$$
$$ \vdots \quad \quad \quad \quad  \vdots \quad \quad \quad \quad \vdots$$
$$u_{P-1} - z (b_{P-1} u_{P-2} + a_{P-1} u_{P-1})  = z u_{P}$$
$$v_{P-1} - z (b_{P-1} v_{P-2} + a_{P-1} v_{P-1}) = z v_{P}$$

Now if we multiply each $u$ equation with $v_P/u_P$ and subtract the corresponding $v$ equation from it, we finaly obtain

$$\left(u_0\frac{v_P}{u_P}  - v_0\right) - z\left[ a_0 \left(u_0\frac{v_P}{u_P} - v_0\right) + \left(u_1\frac{v_P}{u_P} - v_1\right)\right] = s_0$$
$$ \vdots \quad \quad \quad \quad  \vdots \quad \quad \quad \quad \vdots$$
$$\left(u_k\frac{v_P}{u_P} - v_k\right) - z\left[ b_k \left(u_{k-1}\frac{v_P}{u_P} - v_{k-1}\right) + a_k \left(u_k\frac{v_P}{u_P} - v_k\right) + \left(u_{k+1}\frac{v_P}{u_P} - v_{k+1}\right)\right] = 0$$
$$ \vdots \quad \quad \quad \quad  \vdots \quad \quad \quad \quad \vdots$$
$$\left(u_{P-1}\frac{v_P}{u_P} - v_{P-1}\right) - z\left[ b_{P-1} \left(u_{P-2}\frac{v_P}{u_P} - v_{P-2}\right) + a_{P-1} \left(u_{P-1}\frac{v_P}{u_P} - v_{P-1}\right)\right]= 0$$

By defining a vector $(w_0, w_1, \ldots, w_{P-1})$ with components $w_k = u_k v_P/u_P - v_k$ we can write the set of equation in a matrix form

$$({\bf I} - z {\bf J}) (w_0, w_1, \ldots, w_{P-1})^T = s_0 (1, 0, \ldots, 0)^T$$

where the $P\times P$ Jacobi tridiagonal matrix (operator) $\bf J$ is

$$ {\bf J} = \left(\begin{array}{ccccc}
a_0 & 1   &   &  & \\
b_1 & a_1 & 1 &  & \\
0   & b_2 & a_2 & 1 & \\
& &  \ddots & \ddots & \\
& & & b_{P-1} & a_{P-1}
\end{array}\right)$$

If we multiply on the left hand side by the inverse of $({\bf I} - z {\bf J})$ and then by the vector $(1, 0, \ldots, 0)$ we obtain that 

$$w_0 = s_0 (1, 0, \ldots) ({\bf I} - z {\bf J})^{-1} (1, 0, \ldots, 0)^T$$

but 

$$w_0 = u_0 \frac{v_P}{u_P} - v_0 = \frac{q_{-1}}{e_{-2}} \frac{q_{2P-1}(z)}{p_{2P-1}(z)} = R_{2P-1}(z)$$

**Point #6:** The Padé  approximant for a sequence $s_0, s_1,\ldots s_{2P}$ is given by

$$ R_{2P-1}(z) = s_0 (1, 0, \ldots) V^{-1} ({\bf I} - z {\bf \Lambda})^{-1} V (1, 0, \ldots, 0)^T$$

If the matrix ${\bf J}$ is non-defective it has the spectral decomposition ${\bf J} = {\bf V} {\bf \Lambda} {\bf V}^{-1}$, where $\bf \Lambda$ is the eigenvalues diagonal matrix, and $\bf V$ is the matrix with eigenvectors along the columns.
We can finally write

$$R_{2P-1}(z) = s_0 \sum_{j=1}^{P} \frac{V_{1j} V^{-1}_{j1}}{1 - z \lambda_j}$$

which shows that the poles of the Padé approximant are $\zeta_j = 1/\lambda_j$, the reciprocals of the eigenvalues of the J matrix, and the corresponding residues relate to the first component of each eigenvector as: $\rho_j = - s_0 V_{1j} V^{-1}_{j1}/\lambda_j$

The Padé approximant for a signal $s_0, s_1,\ldots, s_{2P}$ is therefore

$$R_{2P-1}(z) = \sum_{j=1}^P \frac{\rho_j}{z - \zeta_j}$$

---------

Let's test this with a signal that has a generating function with two poles:
$$ S(z) = \frac{r_1}{p_1 z - 1} + \frac{r_2}{p_2 z - 1}$$


In [26]:
import sympy as sp
z,r1,p1,r2,p2= sp.symbols('z,r1,p1,r2,p2')
s = {}
P = 2
G = r1/(z*p1-1) + r2/(z*p2-1)
term = sp.series(G,z,0,n=None)
for k in range(2*P):
    s[k]=next(term).coeff(z,k)

In [27]:
S,P = 0,2
q,e = {},{}
for k in range(2*P):
    S += s[k]*z**k
q[-2], q[-1], e[-2], e[-1] = 0,1,1,1
for k in range(2*P):
    e[k] = ((S*q[k-1]).expand()).coeff(z,k)
    q[k] = (((e[k-1]*q[k-1]- z*e[k]*q[k-2])/e[k-2]).expand()).simplify()

In [31]:
J = sp.matrices.zeros(P)
for k in range(P):
    if k == 0:
        J[k,k] = e[-2]//e[-1]*e[1]/e[0]
    else:
        J[k,k] = e[2*k]/e[2*k-1]*e[2*k-3]/e[2*k-2] + e[2*k-2]/e[2*k-1]*e[2*k+1]/e[2*k]
        J[k,k-1]=e[2*k-4]/e[2*k-2]*e[2*k]/e[2*k-2]
    if k != P-1:
        J[k,k+1]=1
J.simplify()

In [32]:
V,D = J.diagonalize()
D.simplify()
V1 = V.inv()
R = 0
for j in range(P):
    R += sp.simplify(s[0]*V[0,j]*V1[j,0]/(1-z*D[j,j]))

In [33]:
R

r1/(p1*z - 1) + r2/(p2*z - 1)

Let's study a numerical example with 5 poles:

In [34]:
import sympy as sp
z = sp.symbols('z')
s={}; F=[]
r1,r2,r3,r4,r5 = 1,2,3,4,5
m1,m2,m3,m4,m5 = 6,7,8,9,10
P = 5
G = r1/(1-z*m1) + r2/(1-z*m2) + r3/(1-z*m3) + r4/(1-z*m4) + r5/(1-z*m5)
term = sp.series(G,z,0,n=None)
for k in range(2*P):
    s[k]=next(term).coeff(z,k)
    F.append(s[k])

In [35]:
S,P = 0,5
q,e = {},{}
for k in range(2*P):
    S += s[k]*z**k
q[-2], q[-1], e[-2], e[-1] = 0,1,1,1
for k in range(2*P):
    e[k] = ((S*q[k-1]).expand()).coeff(z,k)
    q[k] = (((e[k-1]*q[k-1]- z*e[k]*q[k-2])/e[k-2]).expand()).simplify()

In [36]:
J = sp.matrices.zeros(P)
for k in range(P):
    if k == 0:
        J[0,0] = e[-2]//e[-1]*e[2*k+1]/e[2*k]
    else:
        J[k,k] = e[2*k]/e[2*k-1]*e[2*k-3]/e[2*k-2] + e[2*k-2]/e[2*k-1]*e[2*k+1]/e[2*k]
        J[k,k-1]=e[2*k-4]/e[2*k-2]*e[2*k]/e[2*k-2]
    if k != P-1:
        J[k,k+1]=1
J.simplify()

In [37]:
V,D = J.diagonalize()
D.simplify()
V1 = V.inv()
R = 0
for j in range(P):
    R += sp.simplify(s[0]*V[0,j]*V1[j,0]/(1-z*D[j,j]))

The rational function calculated by the algorithm reproduces the original one:

In [38]:
R

-5/(10*z - 1) - 4/(9*z - 1) - 3/(8*z - 1) - 2/(7*z - 1) - 1/(6*z - 1)

If we look at the J matrix, we see that elements are kept ar fractions, even the eigenvalues are calculated in exact arithmetic

In [39]:
J

Matrix([
[26/3,      1,      0,      0,    0],
[14/9, 119/15,      1,      0,    0],
[   0,  36/25, 274/35,      1,    0],
[   0,      0,  54/49, 491/63,    1],
[   0,      0,      0,  50/81, 70/9]])

In [40]:
D

Matrix([
[6, 0, 0, 0,  0],
[0, 7, 0, 0,  0],
[0, 0, 8, 0,  0],
[0, 0, 0, 9,  0],
[0, 0, 0, 0, 10]])

The algorithm in `pade_approx` converts the signal to integer values, obtains the polynomials and $e$ coeffcients with integer coefficient, and the J-matrix elements as fractions, and the end converts those fractions to floating point representation so that to take advantage of existing optimized linear algebra packages. At this point, the numerics are more stable and the risk of losing precision is much reduced, the advantage is a significant increase in speed.

In [41]:
import pade_approx

In [42]:
pade_approx.poles_zeros(F)

(array([10.,  9.,  8.,  7.,  6.]),
 array([5.00000000000001, 3.99999999999997, 3.00000000000000,
        2.00000000000001, 1.00000000000001], dtype=object))

Here is an extreme example with a rational function with 20 poles

In [43]:
n = 20
import sympy as sp
z = sp.symbols('z')
R = [1 + k for k in range(n)]
Z = [n+1 + k for k in range(n)]
G = 0
S = []
for k in range(n):
    G += R[k]/(1-z*Z[k])
term = sp.series(G,z,0,n=None)
for k in range(2*n):
    S.append(int(next(term).coeff(z,k)))

In [44]:
pade_approx.poles_zeros(S)

(array([40.00000001, 38.99999986, 38.00000064, 36.99999831, 36.00000223,
        35.00000067, 33.99999179, 33.00001333, 31.99999663, 30.99997704,
        30.00004684, 28.9999513 , 28.00003114, 26.99998783, 26.00000268,
        24.99999948, 24.00000039, 22.99999977, 22.00000006, 20.99999999]),
 array([20.00000006, 18.99999861, 18.00001092, 16.99995274, 16.00013388,
        14.99972993, 14.00040037, 12.99958107, 12.0002341 , 11.00012263,
         9.99954367,  9.00055662,  7.99959311,  7.00018012,  5.99996592,
         4.99998933,  4.00000943,  2.99999706,  2.00000047,  0.99999997]))

The poles of a sinusoidal signal are complex and complex conjugate to each other

In [45]:
n=512
import numpy as np, matplotlib.pyplot as pl
Factor=10**14
S = np.asarray(np.cos(4*np.pi*np.arange(n)/n )*Factor,dtype=np.int64).tolist()
print(np.exp(1j*4*np.pi/n))

(0.9996988186962042+0.024541228522912288j)


In [46]:
(L,R) = pade_approx.poles_zeros(S)

The poles with the greatest residues are:

In [47]:
L[:2]

array([0.99969882-0.02454123j, 0.99969882+0.02454123j])